# FL Server over Secure RPC

We demonstrate how to launch a gRPC server as a federated learning server with authentication. Consider only one client so that we can launch a server and a client (from another notebook) together.

In [1]:
num_clients = 1

## Load server configurations

In this example, we use the `FedAvg` server aggregation algorithm (while there is only one client for easy demo, the aggregation algorithm does not matter a lot though) and the MNIST dataset by loading the server configurations from `examples/resources/configs/mnist/server_fedavg.yaml`.

In [2]:
from omegaconf import OmegaConf

server_config_file = "../../examples/resources/configs/mnist/server_fedavg.yaml"
server_config = OmegaConf.load(server_config_file)
print(OmegaConf.to_yaml(server_config))

client_configs:
  train_configs:
    trainer: VanillaTrainer
    mode: step
    num_local_steps: 100
    optim: Adam
    optim_args:
      lr: 0.001
    loss_fn_path: ./resources/loss/celoss.py
    loss_fn_name: CELoss
    do_validation: true
    do_pre_validation: true
    metric_path: ./resources/metric/acc.py
    metric_name: accuracy
    use_dp: false
    epsilon: 1
    clip_grad: false
    clip_value: 1
    clip_norm: 1
    train_batch_size: 64
    val_batch_size: 64
    train_data_shuffle: true
    val_data_shuffle: false
  model_configs:
    model_path: ./resources/model/cnn.py
    model_name: CNN
    model_kwargs:
      num_channel: 1
      num_classes: 10
      num_pixel: 28
  comm_configs:
    compressor_configs:
      enable_compression: false
      lossy_compressor: SZ2Compressor
      lossless_compressor: blosc
      error_bounding_mode: REL
      error_bound: 0.001
      param_cutoff: 1024
server_configs:
  scheduler: SyncScheduler
  scheduler_kwargs:
    num_clients: 2
 

💡 It should be noted that configuration fields such as `loss_fn_path`, `metric_path`, and `model_path` are the paths to the corresponding files, so we need to change their relative paths now to make sure the paths point to the right files. 

⚠️ We also need change `num_clients` in `server_configs.scheduler_kwargs` to 1.

In [3]:
server_config.client_configs.train_configs.loss_fn_path = (
    "../../examples/resources/loss/celoss.py"
)
server_config.client_configs.train_configs.metric_path = (
    "../../examples/resources/metric/acc.py"
)
server_config.client_configs.model_configs.model_path = (
    "../../examples/resources/model/cnn.py"
)
server_config.server_configs.scheduler_kwargs.num_clients = num_clients

## Create secure SSL server and authenticator

Secure SSL server requires both *public certificate* and *private key* for data encryption. We have provided a example pair of [certificate](https://github.com/APPFL/APPFL/blob/main/src/appfl/comm/grpc/credentials/localhost.crt) and [key](https://github.com/APPFL/APPFL/blob/main/src/appfl/comm/grpc/credentials/localhost.key) for demonstration. **It should be noted that in practice, you should never share your key to others and keep it secretly**. 

💡 Please check this [tutorial](https://appfl.ai/en/latest/tutorials/examples_ssl.html) for more details on how to generate SSL certificates for securing the gRPC connections in practice.


To enable the SSL channel and use the provided certificate and key, we need to set the following. If the user would like to use his own certificate and key, just change the corresponding field to the file path.

In [4]:
server_config.server_configs.comm_configs.grpc_configs.use_ssl = True
server_config.server_configs.comm_configs.grpc_configs.server_certificate_key = (
    "../../src/appfl/comm/grpc/credentials/localhost.key"
)
server_config.server_configs.comm_configs.grpc_configs.server_certificate = (
    "../../src/appfl/comm/grpc/credentials/localhost.crt"
)

## Setup an authenticator

Now we use a naive authenticator, where the server sets a special token and uses token-match to authenticate the client. 

💡 It should be noted that the naive authenticator is only for easy demonstration and is not really safe in practice to protect your FL experiment. We also provide Globus authenticator, and you can also define your own ones.

In [5]:
server_config.server_configs.comm_configs.grpc_configs.use_authenticator = True
server_config.server_configs.comm_configs.grpc_configs.authenticator = (
    "NaiveAuthenticator"
)
server_config.server_configs.comm_configs.grpc_configs.authenticator_args = {
    "auth_token": "A_SECRET_DEMO_TOKEN"
}

## Start server

Now, we are ready to create the server agent using the `server_config` defined and modified above and start the grpc server.

After launching 🚀 the server, let's go to the notebook to launch the client to talk to the server!

💡 After finishing the FL experiment, you need to manually stop the server.

In [6]:
from appfl.agent import ServerAgent
from appfl.comm.grpc import GRPCServerCommunicator, serve

server_agent = ServerAgent(server_agent_config=server_config)

communicator = GRPCServerCommunicator(
    server_agent,
    logger=server_agent.logger,
    **server_config.server_configs.comm_configs.grpc_configs,
)

serve(
    communicator,
    **server_config.server_configs.comm_configs.grpc_configs,
)

appfl: ✅[2025-01-06 12:04:38,132 server]: Logging to ./output/result_Server_2025-01-06-12-04-38.txt
appfl: ✅[2025-01-06 12:04:50,249 server]: Received GetConfiguration request from client Client1
appfl: ✅[2025-01-06 12:04:50,264 server]: Received GetGlobalModel request from client Client1
appfl: ✅[2025-01-06 12:04:50,275 server]: Received InvokeCustomAction set_sample_size request from client Client1
appfl: ✅[2025-01-06 12:04:54,199 server]: Received UpdateGlobalModel request from client Client1
appfl: ✅[2025-01-06 12:04:54,200 server]: Received the following meta data from Client1:
{'pre_val_accuracy': 15.93,
 'pre_val_loss': 2.30059186820012,
 'round': 1,
 'val_accuracy': 93.11,
 'val_loss': 0.24137536506613444}
appfl: ✅[2025-01-06 12:04:58,212 server]: Received UpdateGlobalModel request from client Client1
appfl: ✅[2025-01-06 12:04:58,213 server]: Received the following meta data from Client1:
{'pre_val_accuracy': 93.11,
 'pre_val_loss': 0.24137536478436486,
 'round': 2,
 'val_accur